<figure>
<img src="../Imagenes/logo-final-ap.png"  width="80" height="80" align="left"/> 
</figure>

# <span style="color:blue"><left>Aprendizaje Profundo</left></span>

# <span style="color:red"><center>Visión por Computadora</center></span>
## <span style="color:red"><center>GANs Condicionales</center></span>

##   <span style="color:blue">Profesores</span>

### Coordinador


- Campo Elías Pardo Turriago, cepardot@unal.edu.co 

### Conferencistas


- Alvaro  Montenegro, PhD, ammontenegrod@unal.edu.co
- Daniel  Montenegro, Msc, dextronomo@gmail.com 
- Oleg Jarma, Estadístico, ojarmam@unal.edu.co 

## <span style="color:blue">Asesora Medios y Marketing digital</span>

- Maria del Pilar Montenegro, pmontenegro88@gmail.com 



## <span style="color:blue">Asistentes</span>



- Nayibe Yesenia Arias, naariasc@unal.edu.co
- Venus Celeste Puertas, vpuertasg@unal.edu.co 

## <span style="color:blue">Introducción</span>

Como se habló anteriormente cuando se mostró [StyleGAN](../Cuadernos/redes_generativas_adversarias.ipynb#StyleGAN), y como indirectamente se demostró cuando generamos [Rostros artificialmente](../Cuadernos/DCGAN.ipynb), en el modelo clásico de Redes Generativas Adversarias tenemos un control muy limitado sobre lo que queremos que se genere. Podemos crear un modelo que genere números a mano, pero luego no podemos decirle que queremos que nos genere un 8. 

Así que, si se busca que la red genere resultados específicos se podría Alimentar datos especializados, lo cual le quita robustez al modelo, o podríamos hacer que la red generadora sea capaz de diferenciar diferentes clases. Logrando así, la capacidad de elegir el tipo de output que queremos obtener.

## <span style="color:blue">Con un vector y con una clase.....</span>

La idea principal de las GANs condicionales es el agregar una nuevo tipo de información tanto en los datos reales, como en los vectores aleatorios de entrada para la generación: Clases. Aunque esto parezca una respuesta obvia, no se usan de la forma que se espera.

A nivel de la base de datos real, la estructura del dataset vuelve a ser más clásica. Hacemos la separación clásica entre los labels de interés, mientras se agrega la clase "verdadero" a cada uno. En la generación de bases de datos, además del vector aleatorio que se convertirá en nuestra imagen, le entregamos una de estas clases, dándole una "condición" sobre el tipo de dato que debería estar generando para obtener mejores resultados. De forma resumida, tomamos las clases, usualmente tomadas como la variable respuesta, como una nueva variable de observación

¿Cómo cambian nuestras redes en este caso?

- La red discriminadora gana una buena ventaja, ya que puede discernir la falsedad de una imagen si su clase no coincide. Así que, el discriminador debe ser capaz de:
    - Aceptar el par imagen-clase reales
    - discriminar el par imagen-clase falsos, con clase correcta
    - discriminar el par imagen-clase falsos, con clase incorrecta

- La red generadora, en este nuevo caso, ya no es suficiente el hacer la imagen más realista posible. ahora tiene que ser consciente del tipo de imagen que le estan pidiendo. 
<figure>
<center>
<img src="../Imagenes/cgan.png" width="500" height="450" align="center"/> 
</center>
</figure>

Como podemos apreciar, este cambio es uno bastante sencillo de aplicar, que mejora la convergencia de la red, y le da más opciones al usuario. Razón por la cual se ha tomado como un estándar o una recomendación a la hora de desarrollar GANs

## <span style="color:blue">Generación de Piedra, Papel o Tijeras</span>

Tomaremos el dataset RPM, el cual ya hemos trabajado en [Transfer Learning](../Cuadernos/taller_transfer_learning.ipynb), usaremos la arquitectura profunda de [DCGANs](../Cuadernos/DCGAN.ipynb), y la acondicionaremos a un mejor resultado

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision.utils import save_image
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip \
    -O /mnt/storage/Datasets/rps/rps.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip \
    -O /mnt/storage/Datasets/rps/rps-test-set.zip

In [ ]:
import os
import zipfile

local_zip = '/mnt/storage/Datasets/rps/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/mnt/storage/Datasets/rps')
zip_ref.close()
local_zip = '/mnt/storage/Datasets/rps/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip)
zip_ref.extractall('/mnt/storage/Datasets/rps')
zip_ref.close()